In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

from pytrends.request import TrendReq

## Идеи:

* Взять отношение Media attention index по дням и оценивать функцию от него

In [ ]:
eth = torch.tensor(btc_eth_df.eth.values)
btc = torch.tensor(btc_eth_df.btc.values)

log_eth = torch.tensor(log_btc_eth_df.log_eth.values)
log_btc = torch.tensor(log_btc_eth_df.log_btc.values)

In [ ]:
# Сформируем матрицу X
X = torch.empty(n - h + 1, h)
for i in range(n - h + 1):
    X[i] = log_btc[i:h + i]

X = pd.DataFrame(X)
X.index = log_btc_eth_df.index[h - 1:]

columns = []
for i in range(h, 0, -1):
    columns.append(f'M_t_{i}')
    
X.columns = columns

X_index = X.index
X = X.iloc[:-1, :]
X.index = X_index[1:]

In [ ]:
log_eth_delta = pd.DataFrame(log_btc_eth_df.log_eth[h - 1:] - log_btc_eth_df.log_eth[h - 1:].shift(1)).iloc[1:, :]
log_eth_delta.columns = ['log_eth_delta']

target = log_eth_delta / np.sqrt(delta)
target.columns = ['target']

In [ ]:
# z: Z_k ~ N(0, 1)
torch.manual_seed(222)
Z = torch.distributions.normal.Normal(loc=0, scale=1).sample(sample_shape=torch.Size([n, 1]))
Z_curr = pd.DataFrame(index=X.index, data=Z[h:], columns=['Z_t'])

In [ ]:
data = pd.concat([X, Z_curr, target], axis=1)
data.to_excel('./Data/data.xlsx')
data